<a href="https://colab.research.google.com/github/DongDong-Zoez/pytorchAI/blob/main/Transformer/Swin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Swin Transformer

ViT 這篇論文是結合 CV 和 NLP 兩個領域的一個里程碑模型，但是 ViT 只做了分類任務，把其他的下游任務留給後人去探索。在 2021 年，Swin Transformer 橫空出世，利用 Transformer 的架構一路屌打了現有所有的 CNN 模型，不管是在目標檢測或者語意分割上都完虐了現有 CNN 模型，我們就來看一看 Swin Transformer 到底在 ViT 上做了什麼改變

## Challenges

從 NLP 轉戰 CV 可不是隨口說說的，CV 的特徵要比 NLP 複雜的多 (當然 1D 怎麼能跟 2D 比)。我們在將影像轉為 token 會面臨兩個問題

1. 多尺度特徵，我們知道在一張影像會有密密麻麻的物件，這些物件大小不同。例如貓和車是完全不同大小的兩個物件，如果用 NLP 的 token 去看，就不會有物件大小不同的問題
2. 物件解析度，現在的影像通常都有很大的解析度，如果直接改刀成 patch，會影響計算速度

## Dense Prediction

CNN 架構與 transformer 架構有一個很大的不同，CNN 架構是個層級式的網路，透過網路的池化層越做越深，能夠抓到更高級的特徵，同時網路看到的範圍也越廣 (感受野越大)，但是 transformer 就沒有這個機制，這也是為什麼 ViT 只能夠做圖片分類任務

在密集預測的任務中，我們會需要有物件層級式的訊息，這也是會什麼在物件偵測或語意分割中我們會需要 FPN、SPP、ASPP 等方法。例如物件偵測的模型中，模型一開始的層由於感受野較小，所以多 focus 在小物件上，當模型越來越深後，層的感受野增大了，這時候就可以很好的抓住大物件

Swin Transformer 提出了一個基於 window 的 Attention 運算和 Patch Merging 解決物件解析度問題與多使度特徵問題


## Model Architecture

![](https://user-images.githubusercontent.com/24825165/121768619-038e6d80-cb9a-11eb-8cb7-daa827e7772b.png)

我們可以看到在 Swin Transformer 有四個主要模塊，Patch Partition、Linear Embedding、Swin Transfoermer Block 和 Patch Merging

- Patch Partition: 同 ViT 裡對 Patch 分割為不重疊的小塊
- Linear Embedding: 就是一個 transformation，將輸入投射到我們想要的維度
- Swin Transformer Block: 同 (c)，會先進行 W-MSA 在進行 SW-MSA
- Patch Merging: Pixel Shuffle 的反操作

### Patch Merging

我們先重點看一下圖 (a)，圖 (a) 也是 Swin Transformer 的核心，他利用 Patch Merging 達到層級式特徵的效果

以下是 Patch Merging 的操作流程

![](https://aitechtogether.com/wp-content/uploads/2022/02/1645189655-1deacff3ea7349088465a65f35a60718.webp)

The image copy from [https://aitechtogether.com/wp-content/uploads/2022/02/1645189655-1deacff3ea7349088465a65f35a60718.webp](https://aitechtogether.com/wp-content/uploads/2022/02/1645189655-1deacff3ea7349088465a65f35a60718.webp)

簡單來說就是把鄰近點都 Merge 起來，之後在對 channel 的維度做連接，因為要仿照 CNN 架構中的通道數大小，所以會通過一個簡單的 1*1 Convolution 去將通道數減半 (ResNet 或 VGG 等方法的通道數都是倍數增長 (64 --> 128 --> 256))

$$
H\times W\times C \to \frac{1}{2}H\times \frac{1}{2}W \times C \to \frac{1}{2}H\times \frac{1}{2}W \times 4C \to \frac{1}{2}H\times \frac{1}{2}W \times 2C
$$

整個過程把影像長寬減半，但是通道數翻倍，類似於影像中的**池化**計算

這個簡單的 Patch Merging 就可以達到層級特徵的效果以及減少計算

### Calculate Unit

不同於 ViT，Swin Transformer 會把原圖分成 $\frac{h}{\text{path_size}\times M}\times\frac{w}{\text{path_size}\times M}$ 個窗口，每個窗口包含 $M\times M$ 個 patch，在計算 Attention 時只在窗口中計算，這樣就可以大大減少序列長度


### Swin Tranformer Block

前面提到用窗口做為 Attention 的計算單元，但是這樣又衍生了一個問題，Attention 的初衷是為了計算全局的信息，如果直接在窗口進行 Attention，這樣窗口之間沒有連動，也不會進行信息的交換

所以作者提出了移動窗口的想法，同圖 (b)，我們會把原本的窗口劃分線往右下移動，這時候我們得到 9 個區塊，與原始的 4 個區塊不同，注意到如果直接執行 Attention 運算的話會導致運算量翻倍，而且窗口之間大小不一，所以作者提出了一個 mask 方法

![](https://miro.medium.com/max/1400/1*IpqGMZnwmp9p68P60yW8CQ.png)

The image copy from [https://miro.medium.com/max/1400/1*IpqGMZnwmp9p68P60yW8CQ.png](https://miro.medium.com/max/1400/1*IpqGMZnwmp9p68P60yW8CQ.png)

我們會先把左上方的窗口移到右下方，之後進行 mask 的 Attention 運算，最後在返回

為了方便介紹 mask 運算，我們看下面這張圖

![](https://user-images.githubusercontent.com/13436512/157567931-53a3f547-a2c0-44b6-bd4c-579f43dc9008.png)

The iamge copy from [https://user-images.githubusercontent.com/13436512/157567931-53a3f547-a2c0-44b6-bd4c-579f43dc9008.png](https://user-images.githubusercontent.com/13436512/157567931-53a3f547-a2c0-44b6-bd4c-579f43dc9008.png)

我們先看下面這張圖的 window2，window2 包含了 3、6 區塊，其中 6 區塊是其他地方移過來的，那麼 window2 就不應該直接做 Attention，你可以想像如果 3 區塊代表地面 6 區塊代表天空，那麼直接做 Attention 很不合理，所以作者用了一個方法，先把每個 window2 的從左上沿著右邊把向量拉直，將拉直的向量乘上他的轉置，我們就會得到一個矩陣，這個矩陣內只有 3 區塊與 3 區塊元素相乘的位置或者 6 區塊與 6 區塊位置的地方需要計算 Attetion，所以我們把其他位置 mask 掉，也就是加上一個很大的負值，使其通過 softmax 時為 0，不同窗口對應的 mask 如同右手邊所示

Swin Transformer Block 都會先計算基於窗口的 Attention，再算基於移動窗口的 Attention

### Forward Pass

接下來我們看看整個模型的前向過程，首先作者用了 patch size 為 4 去對影像作分割，之後通過線性映射到 C 維度，之後經過 Swin Transformer 計算 (同 Attention 計算，會保持輸入與輸出維度一致)，之後每次都會通過一個 Patch Merging 的計算會把影像長寬減半，通道翻倍

其流程用數學符號表示如下:

$$
\begin{align}
&\hat z^l=\mbox{W-MSA}(\mbox{LN}(z^{l-1}))+z^{l-1}, \\
&z^l=\mbox{MLP}(\mbox{LN}(\hat z^{l}))+\hat z^{l}, \\
&\hat z^{l+1}=\mbox{SW-MSA}(\mbox{LN}(z^{l}))+z^{l}, \\
&z^{l+1}=\mbox{W-MSA}(\mbox{LN}(\hat z^{l+1}))+\hat z^{l+1}, \\
\end{align}
$$

## Dissicussion

Swin Transformer 的名字來自於其提出的運算 S (Shifted) + win (windows)，其借鑑了很多 CNN 模型的架構做參考，我們可以看成把 CNN 架構中的 Convolution 運算換成 transformer 運算就是 Swin Transformer，也可以說是披著 Transformer 皮的 CNN 架構

In [87]:
#!pip install timm
import timm
import torch
x = torch.rand(1,3,224,224)
m = timm.models.swin_base_patch4_window7_224()
print(m.patch_embed(x).shape)
m(x).shape

torch.Size([1, 3136, 128])


torch.Size([1, 1000])

In [88]:
import torch.nn as nn

def to_2tuple(x):
    return (x, x)

class HybridEmbed(nn.Module):
    """ CNN Feature Map Embedding
    Extract feature map from CNN, flatten, project to embedding dim.
    """
    def __init__(self, backbone, img_size=224, patch_size=1, feature_size=None, in_chans=3, embed_dim=768):
        super().__init__()
        assert isinstance(backbone, nn.Module)
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.backbone = backbone
        if feature_size is None:
            with torch.no_grad():
                # NOTE Most reliable way of determining output dims is to run forward pass
                training = backbone.training
                if training:
                    backbone.eval()
                o = self.backbone(torch.zeros(1, in_chans, img_size[0], img_size[1]))
                if isinstance(o, (list, tuple)):
                    o = o[-1]  # last feature if backbone outputs list/tuple of features
                feature_size = o.shape[-2:]
                feature_dim = o.shape[1]
                backbone.train(training)
        else:
            feature_size = to_2tuple(feature_size)
            if hasattr(self.backbone, 'feature_info'):
                feature_dim = self.backbone.feature_info.channels()[-1]
            else:
                feature_dim = self.backbone.num_features
        assert feature_size[0] % patch_size[0] == 0 and feature_size[1] % patch_size[1] == 0
        self.grid_size = (feature_size[0] // patch_size[0], feature_size[1] // patch_size[1])
        self.num_patches = self.grid_size[0] * self.grid_size[1]
        self.proj = nn.Conv2d(feature_dim, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.backbone(x)
        if isinstance(x, (list, tuple)):
            x = x[-1]  # last feature if backbone outputs list/tuple of features
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x

In [89]:
class HybridEmbed(nn.Module):
    """ CNN Feature Map Embedding
    Extract feature map from CNN, flatten, project to embedding dim.
    """
    def __init__(self, backbone, img_size=224, patch_size=1, feature_size=None, in_chans=3, embed_dim=768):
        super().__init__()
        assert isinstance(backbone, nn.Module)
        img_size = (img_size, img_size)
        patch_size = (patch_size, patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.backbone = backbone
        if feature_size is None:
            with torch.no_grad():
                # NOTE Most reliable way of determining output dims is to run forward pass
                training = backbone.training
                if training:
                    backbone.eval()
                o = self.backbone(torch.zeros(1, in_chans, img_size[0], img_size[1]))
                if isinstance(o, (list, tuple)):
                    o = o[-1]  # last feature if backbone outputs list/tuple of features
                feature_size = o.shape[-2:]
                feature_dim = o.shape[1]
                backbone.train(training)
        else:
            feature_size = (feature_size, feature_size)
            if hasattr(self.backbone, 'feature_info'):
                feature_dim = self.backbone.feature_info.channels()[-1]
            else:
                feature_dim = self.backbone.num_features
        assert feature_size[0] % patch_size[0] == 0 and feature_size[1] % patch_size[1] == 0
        self.grid_size = (feature_size[0] // patch_size[0], feature_size[1] // patch_size[1])
        self.num_patches = self.grid_size[0] * self.grid_size[1]
        self.proj = nn.Conv2d(feature_dim, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.backbone(x)
        if isinstance(x, (list, tuple)):
            x = x[-1]  # last feature if backbone outputs list/tuple of features
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x

In [90]:
class HybridSwin(nn.Module):

    def __init__(self, backbone, embedder, img_size=224, embed_dim=128, num_classes=10):
        super().__init__()
        assert isinstance(backbone, nn.Module)
        assert isinstance(embedder, nn.Module)
        self.backbone = backbone
        self.embedder = embedder
        self.backbone.patch_embed = HybridEmbed(self.embedder, img_size=img_size, embed_dim=embed_dim)
        print(self.backbone.patch_embed(x).shape)
        self.in_features = self.backbone.head.in_features
        self.backbone.head = nn.Linear(self.in_features, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        return x

注意到我們有影像輸入大小為 224$\times$224$\times$3，分割成 8$\times$8 個窗口，每個窗口包含 7$\times$7 ($M$=7) 個 patch，總共有 56$\times$56 個 patch，每個 patch 大小為 4$\times$4$\times$3

為了達到 HyBrid 架構，我們必須讓特徵圖的長寬也限制為 56$\times$56

In [92]:
x = torch.randn(1,3,448,448)
e = timm.create_model('tf_efficientnet_b4_ns', features_only=True, out_indices=[2])
h = HybridSwin(m, e)
h(x).shape

torch.Size([1, 3136, 128])


torch.Size([1, 10])